# Contextual RAG System with Hybrid Search and Reranking

<!-- ![](./images/contextual_rag_flow_chart.png | width=100) -->
<img src="./images/contextual_rag_flow_chart.png" width="800">

## Install OpenAI, and LangChain dependencies

In [ ]:
!pip install langchain==0.3.4
!pip install langchain-openai==0.2.3
!pip install langchain-community==0.3.3
!pip install jq==1.8.0
!pip install pymupdf==1.24.12
!pip install httpx==0.27.2

## Install Chroma Vector DB LangChain wrapper

In [ ]:
!pip install langchain-chroma==0.1.4

## Install BM25 dependencies

In [ ]:
!pip install rank_bm25==0.2.2

## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Setup Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

## Loading and Processing the Data

### Get the dataset

In [ ]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1aZxZejfteVuofISodUrY2CDoyuPLYDGZ download it
# manually upload it on colab
!gdown 1aZxZejfteVuofISodUrY2CDoyuPLYDGZ

Downloading...
From: https://drive.google.com/uc?id=1aZxZejfteVuofISodUrY2CDoyuPLYDGZ
To: /content/rag_docs.zip
100% 5.92M/5.92M [00:00<00:00, 111MB/s]


In [ ]:
!unzip rag_docs.zip

Archive:  rag_docs.zip
   creating: rag_docs/
  inflating: rag_docs/attention_paper.pdf  
  inflating: rag_docs/cnn_paper.pdf  
  inflating: rag_docs/resnet_paper.pdf  
  inflating: rag_docs/vision_transformer.pdf  
  inflating: rag_docs/wikidata_rag_demo.jsonl  


### Load and Process JSON Documents

In [4]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(file_path='./rag_docs/wikidata_rag_demo.jsonl',
                    jq_schema='.',
                    text_content=False,
                    json_lines=True)
wiki_docs = loader.load()

In [ ]:
len(wiki_docs)

1801

In [ ]:
wiki_docs[3]

Document(metadata={'source': '/content/rag_docs/wikidata_rag_demo.jsonl', 'seq_num': 4}, page_content='{"id": "71548", "title": "Chi-square distribution", "paragraphs": ["In probability theory and statistics, the chi-square distribution (also chi-squared or formula_1\\u00a0 distribution) is one of the most widely used theoretical probability distributions. Chi-square distribution with formula_2 degrees of freedom is written as formula_3. It is a special case of gamma distribution.", "Chi-square distribution is primarily used in statistical significance tests and confidence intervals. It is useful, because it is relatively easy to show that certain probability distributions come close to it, under certain conditions. One of these conditions is that the null hypothesis must be true. Another one is that the different random variables (or observations) must be independent of each other."]}')

In [ ]:
import json
from langchain.docstore.document import Document    # Class for storing a piece of text and associated Metadata

wiki_docs_processed = []

for doc in wiki_docs:
    doc = json.loads(doc.page_content)
    metadata = {
        "title": doc['title'],
        "id": doc['id'],
        "source": "Wikipedia",
        "page": 1
    }
    data = ' '.join(doc['paragraphs'])
    wiki_docs_processed.append(Document(page_content=data, metadata=metadata))

In [ ]:
wiki_docs_processed[3]

Document(metadata={'title': 'Chi-square distribution', 'id': '71548', 'source': 'Wikipedia', 'page': 1}, page_content='In probability theory and statistics, the chi-square distribution (also chi-squared or formula_1\xa0 distribution) is one of the most widely used theoretical probability distributions. Chi-square distribution with formula_2 degrees of freedom is written as formula_3. It is a special case of gamma distribution. Chi-square distribution is primarily used in statistical significance tests and confidence intervals. It is useful, because it is relatively easy to show that certain probability distributions come close to it, under certain conditions. One of these conditions is that the null hypothesis must be true. Another one is that the different random variables (or observations) must be independent of each other.')

### Load and Process PDF documents

#### Create Chunk Contexts for Contextual Retrieval

![](https://i.imgur.com/cqJqEv0.png)


- Prepend chunk-specific explanatory context to each chunk before creating the vector DB embeddings and TF-IDF vectors.
- Helps with having keywords or phrases in each chunk based on its relevance to the overall document.
- Improves retrieval performance quite a bit, which also helps with the overall RAG generation results because of better context.
- The contextual chunking prompt can be built in various ways depending on your use-case.

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [5]:
# create Chunk Context generation chain:

from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser


def generate_chunk_context(document, chunk):

    chunk_process_prompt = """You are an AI assistant specializing in research paper analysis.
                            Your task is to provide brief, relevant context for a chunk of text
                            based on the following research paper.

                            Here is the research paper:
                            <paper>
                            {paper}
                            </paper>

                            Here is the chunk we want to situate within the whole document:
                            <chunk>
                            {chunk}
                            </chunk>

                            Provide a concise context (3-4 sentences max) for this chunk,
                            considering the following guidelines:

                            - Give a short succinct context to situate this chunk within the overall document
                            for the purposes of improving search retrieval of the chunk.
                            - Answer only with the succinct context and nothing else.
                            - Context should be mentioned like 'Focuses on ....'
                            do not mention 'this chunk or section focuses on...'

                            Context:
                        """

    prompt_template = ChatPromptTemplate.from_template(chunk_process_prompt)

    agentic_chunk_chain = (prompt_template
                                |
                               llm
                                |
                            StrOutputParser())

    context = agentic_chunk_chain.invoke({'paper': document, 'chunk': chunk})

    return context

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

def create_contextual_chunks(file_path, chunk_size=3500, chunk_overlap=0):

    print('Loading pages:', file_path)
    loader = PyMuPDFLoader(file_path)       # load text from PDF
    doc_pages = loader.load()

    print('Chunking pages:', file_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,            
                                              chunk_overlap=chunk_overlap)
    doc_chunks = splitter.split_documents(doc_pages)                            # Split into Chunks | also adds Metadata to each Chunk: like [Page Number], [Source Doc Name]

    print('Generating contextual chunks:', file_path)
    original_doc = '\n'.join([doc.page_content for doc in doc_chunks])

    contextual_chunks = []
    
    for chunk in doc_chunks:
        chunk_content = chunk.page_content
        chunk_metadata = chunk.metadata
        chunk_metadata_upd = {
            'id': str(uuid.uuid4()),                # create unique id for the chunk
            'page': chunk_metadata['page'],         # page number of the chunk in the source doc
            'source': chunk_metadata['source'],     # 
            'title': chunk_metadata['source'].split('/')[-1]    # source doc title
        }

        context = generate_chunk_context(original_doc, chunk_content)
        contextual_chunks.append(Document(page_content=context+'\n'+chunk_content,      # Prepend Chunk Context to the Chunk & add Metadata 
                                          metadata=chunk_metadata_upd))
    print('Finished processing:', file_path)
    print()
    return contextual_chunks

In [ ]:
from glob import glob

pdf_files = glob('./rag_docs/*.pdf')
pdf_files

['./rag_docs/resnet_paper.pdf',
 './rag_docs/cnn_paper.pdf',
 './rag_docs/vision_transformer.pdf',
 './rag_docs/attention_paper.pdf']

In [ ]:
paper_docs = []

for fp in pdf_files:
    paper_docs.extend(create_contextual_chunks(file_path=fp, chunk_size=3500))

Loading pages: ./rag_docs/resnet_paper.pdf
Chunking pages: ./rag_docs/resnet_paper.pdf
Generating contextual chunks: ./rag_docs/resnet_paper.pdf
Finished processing: ./rag_docs/resnet_paper.pdf

Loading pages: ./rag_docs/cnn_paper.pdf
Chunking pages: ./rag_docs/cnn_paper.pdf
Generating contextual chunks: ./rag_docs/cnn_paper.pdf
Finished processing: ./rag_docs/cnn_paper.pdf

Loading pages: ./rag_docs/vision_transformer.pdf
Chunking pages: ./rag_docs/vision_transformer.pdf
Generating contextual chunks: ./rag_docs/vision_transformer.pdf
Finished processing: ./rag_docs/vision_transformer.pdf

Loading pages: ./rag_docs/attention_paper.pdf
Chunking pages: ./rag_docs/attention_paper.pdf
Generating contextual chunks: ./rag_docs/attention_paper.pdf
Finished processing: ./rag_docs/attention_paper.pdf



In [ ]:
len(paper_docs)

79

In [ ]:
paper_docs[0]

Document(metadata={'id': 'd5c90113-2421-42c0-bf09-813faaf75ac7', 'page': 0, 'source': './rag_docs/resnet_paper.pdf', 'title': 'resnet_paper.pdf'}, page_content='Focuses on the introduction of a residual learning framework designed to facilitate the training of significantly deeper neural networks, addressing challenges such as vanishing gradients and degradation of accuracy. It highlights the empirical success of residual networks, particularly their performance on the ImageNet dataset and their foundational role in winning multiple competitions in 2015.\nDeep Residual Learning for Image Recognition\nKaiming He\nXiangyu Zhang\nShaoqing Ren\nJian Sun\nMicrosoft Research\n{kahe, v-xiangz, v-shren, jiansun}@microsoft.com\nAbstract\nDeeper neural networks are more difﬁcult to train. We\npresent a residual learning framework to ease the training\nof networks that are substantially deeper than those used\npreviously. We explicitly reformulate the layers as learn-\ning residual functions with

### Combine all document chunks in one list

In [ ]:
len(wiki_docs_processed)

1801

In [ ]:
total_docs = wiki_docs_processed + paper_docs
len(total_docs)

1880

## Index Document Chunks and Embeddings in Vector DB

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [ ]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

### Vector DB Indexing for Semantic Search

In [ ]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=total_docs,
                                  collection_name='my_context_db',
                                  embedding=openai_embed_model,
                                  # need to set the Distance function to Cosine else it uses Euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_context_db")              # set custom disk location

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [ ]:
# load from disk
chroma_db = Chroma(persist_directory="./my_context_db",
                   collection_name='my_context_db',
                   embedding_function=openai_embed_model)

In [ ]:
chroma_db

### Semantic Similarity based Retrieval

We use simple cosine similarity here and retrieve the top 5 similar documents based on the user input query

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

### BM25 Indexing for Keyword based Retrieval

In [ ]:
from langchain.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents=total_docs,
                                              k=5)
bm25_retriever

BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7e84015b33a0>, k=5)

## Build Retrieval Strategy

### Build Base Ensemble Retriever

In [ ]:
from langchain.retrievers import EnsembleRetriever

# Reciprocal Rank Fusion:

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, similarity_retriever],      # Vector Similarity Search + BM25 Search
    weights=[0.5, 0.5]
)
ensemble_retriever

EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7e84015b33a0>, k=5), VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7e84015b1330>, search_kwargs={'k': 5})], weights=[0.5, 0.5])

### Chained Retrieval with Reranker

In [ ]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

# download an open-source Re-Ranker Model - BAAI/bge-reranker-v2-m3
reranker = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
reranker_compressor = CrossEncoderReranker(model=reranker, top_n=5)

# Retriever 2 - Uses a Reranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(
    base_retriever=ensemble_retriever,              # Stage 1: Base Retriever: (Vector Similarity Search + BM25 Search) : Reciprocal Rank Fusion
    base_compressor=reranker_compressor             # Stage 2: Re-ranker
)
final_retriever

ContextualCompressionRetriever(base_compressor=CrossEncoderReranker(model=HuggingFaceCrossEncoder(client=<sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder object at 0x7e82f6796710>, model_name='BAAI/bge-reranker-v2-m3', model_kwargs={}), top_n=5), base_retriever=EnsembleRetriever(retrievers=[BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x7e84015b33a0>, k=5), VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7e84015b1330>, search_kwargs={'k': 5})], weights=[0.5, 0.5]))

In [ ]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()

In [ ]:
query = "what is machine learning?"
top_docs = final_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'id': '564928', 'page': 1, 'source': 'Wikipedia', 'title': 'Machine learning'}
Content Brief:


Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.


Metadata: {'id': '663523', 'page': 1, 'source': 'Wikipedia', 'title': 'Deep learning'}
Content Brief:


Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human brain) in many ways, whic


Metadata: {'id': '359370', 'page': 1, 'source': 'Wikipedia', 'title': 'Supervised learning'}
Content Brief:


In machine learning, supervised learning is the task of inferring a function from labelled training data. The results of the training are known beforehand, the system simply learns how to get to these results correctly. Usually, such systems work with vectors. They get the training data and the result of the training as two vectors and produce a "classifier". Usually, the system uses inductive reasoning to generalize the training data.


Metadata: {'id': '44742', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial neural network'}
Content Brief:


A neural network (also called an ANN or an artificial neural network) is a sort of computer software, inspired by biological neurons. Biological brains are capable of solving difficult problems, but each neuron is only responsible for solving a very small part of the problem. Similarly, a neural network is made up of cells that work together to produce a desired result, although each individual cell is only responsible for solving a small part of the problem. This is one method for creating artificially intelligent programs. Neural networks are an example of machine learning, where a program can change as it learns to solve a problem. A neural network can be trained and improved with each example, but the larger the neural network, the more examples it needs to perform well—often needing millions or billions of examples in the case of deep learning. There are two ways to think of a neural network. First is like a human brain. Second is like a mathematical equation.


Metadata: {'id': 'fc4c7a16-11e3-4dd4-9e5f-8152dc481836', 'page': 0, 'source': './rag_docs/cnn_paper.pdf', 'title': 'cnn_paper.pdf'}
Content Brief:


Focuses on the introduction of Convolutional Neural Networks (CNNs) within the broader field of Artificial Neural Networks (ANNs), highlighting their significance in image-driven pattern recognition tasks. It outlines the foundational concepts of ANNs, their architecture, and the evolution of machine learning techniques, setting the stage for a deeper exploration of CNNs and their applications.
An Introduction to Convolutional Neural Networks
Keiron O’Shea1 and Ryan Nash2
1 Department of Computer Science, Aberystwyth University, Ceredigion, SY23 3DB
keo7@aber.ac.uk
2 School of Computing and Communications, Lancaster University, Lancashire, LA1
4YW
nashrd@live.lancs.ac.uk
Abstract. The ﬁeld of machine learning has taken a dramatic twist in re-
cent times, with the rise of the Artiﬁcial Neural Network (ANN). These
biologically inspired computational models are able to far exceed the per-
formance of previous forms of artiﬁcial intelligence in common machine
learning tasks. One of the mos

In [ ]:
query = "what is the difference between transformers and vision transformers?"
top_docs = final_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'id': '634ee903-03bc-42d1-ba48-615254cf9820', 'page': 0, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the introduction of the Vision Transformer (ViT) model, which applies a pure Transformer architecture to image classification tasks by treating image patches as tokens. It highlights the limitations of traditional convolutional networks in computer vision and presents the advantages of using Transformers, particularly when pre-trained on large datasets, achieving competitive results with reduced computational resources.
Published as a conference paper at ICLR 2021
AN IMAGE IS WORTH 16X16 WORDS:
TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE
Alexey Dosovitskiy∗,†, Lucas Beyer∗, Alexander Kolesnikov∗, Dirk Weissenborn∗,
Xiaohua Zhai∗, Thomas Unterthiner, Mostafa Dehghani, Matthias Minderer,
Georg Heigold, Sylvain Gelly, Jakob Uszkoreit, Neil Houlsby∗,†
∗equal technical contribution, †equal advising
Google Research, Brain Team
{adosovitskiy, neilhoulsby}@google.com
ABSTRACT
While the Transformer architecture has become the de-facto standard for natural
language processing tasks, i


Metadata: {'id': 'b896c93d-6330-421c-a236-af9437e9c725', 'page': 1, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the performance of the Vision Transformer (ViT) in comparison to convolutional neural networks (CNNs), highlighting the advantages of large-scale training on datasets like ImageNet-21k and JFT-300M. It discusses how ViT achieves state-of-the-art results in image recognition benchmarks despite lacking certain inductive biases inherent to CNNs. Additionally, it references related work on self-attention mechanisms and their application in computer vision.
Published as a conference paper at ICLR 2021
inherent to CNNs, such as translation equivariance and locality, and therefore do not generalize well
when trained on insufﬁcient amounts of data.
However, the picture changes if the models are trained on larger datasets (14M-300M images). We
ﬁnd that large scale training trumps inductive bias. Our Vision Transformer (ViT) attains excellent
results when pre-trained at sufﬁcient scale and transferred to tasks with fewer datapoints. When
pre-trained on the public ImageNet-21k dataset 


Metadata: {'id': '45cb4930-8e97-42f7-a1e8-17d525f25118', 'page': 7, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on a controlled scaling study of various models, including Vision Transformers and ResNets, evaluating their transfer performance from the JFT-300M dataset. It highlights the performance versus pre-training cost, revealing that Vision Transformers generally outperform ResNets in terms of efficiency and scalability, while also discussing the implications for future model scaling efforts.
Published as a conference paper at ICLR 2021
4.4
SCALING STUDY
We perform a controlled scaling study of different models by evaluating transfer performance from
JFT-300M. In this setting data size does not bottleneck the models’ performances, and we assess
performance versus pre-training cost of each model. The model set includes: 7 ResNets, R50x1,
R50x2 R101x1, R152x1, R152x2, pre-trained for 7 epochs, plus R152x2 and R200x3 pre-trained
for 14 epochs; 6 Vision Transformers, ViT-B/32, B/16, L/32, L/16, pre-trained for 7 epochs, plus
L/16 and H/14 pre-trained for 14 epochs; and 5 hybrids, R50+ViT


Metadata: {'id': 'd3a31c54-e58a-4b69-92f4-6cd180c3377d', 'page': 2, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the architecture and methodology of the Vision Transformer (ViT), detailing how images are processed by splitting them into patches, embedding these patches, and utilizing a standard Transformer encoder for image classification tasks. It also describes the integration of position embeddings and the classification head within the model framework.
Published as a conference paper at ICLR 2021
Transformer Encoder
MLP 
Head
Vision Transformer (ViT)
*
Linear Projection of Flattened Patches
* Extra learnable
     [ cl ass]  embedding
1
2
3
4
5
6
7
8
9
0
Patch + Position 
Embedding
Class
Bird
Ball
Car
...
Embedded 
Patches
Multi-Head 
Attention
Norm
MLP
Norm
+
L x
+
Transformer Encoder
Figure 1: Model overview. We split an image into ﬁxed-size patches, linearly embed each of them,
add position embeddings, and feed the resulting sequence of vectors to a standard Transformer
encoder. In order to perform classiﬁcation, we use the standard approach of adding an extra learnable
“classiﬁc


Metadata: {'id': 'fcceed86-4540-40e9-9a64-cdec9ae0da4f', 'page': 7, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the behavior of attention mechanisms in the Vision Transformer (ViT) architecture, highlighting how attention distances vary across layers and the implications for feature extraction. It compares the attention patterns in ViT to those in hybrid models that incorporate convolutional networks, suggesting that early layers in ViT exhibit localized attention similar to CNNs. Additionally, it notes the increase in attention distance with deeper layers, indicating a shift towards more global feature integration relevant for classification tasks.
have consistently small attention distances in the low layers. This highly localized attention is
less pronounced in hybrid models that apply a ResNet before the Transformer (Figure 7, right),
suggesting that it may serve a similar function as early convolutional layers in CNNs. Further, the
attention distance increases with network depth. Globally, we ﬁnd that the model attends to image
regions that are semantically relevant for classiﬁca

## Build the RAG Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):    # formats the document content into a structured string
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (final_retriever           # Revokes the Retriever using the user Query
                      |
                    format_docs),             # formats the document content into a structured string
        "question": RunnablePassthrough()     # to directly pass the user Query without any modifications
    }
      |
    rag_prompt_template
      |
     llm
)

In [ ]:
from IPython.display import display, Markdown

query = "What is machine learning?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Machine learning is a subfield of computer science that provides computers the ability to learn without being explicitly programmed. The concept was introduced by Arthur Samuel in 1959 and is rooted in artificial intelligence. Machine learning focuses on the study and construction of algorithms that can learn from data and make predictions or decisions based on that data. These algorithms follow programmed instructions but can also adapt and improve their performance by building models from sample inputs.

Machine learning is particularly useful in scenarios where designing and programming explicit algorithms is impractical. Some common applications of machine learning include:

- Spam filtering
- Detection of network intruders or malicious insiders
- Optical character recognition (OCR)
- Search engines
- Computer vision

Within the realm of machine learning, there is a subset known as deep learning, which primarily utilizes certain types of neural networks. Deep learning involves learning sessions that can be unsupervised, semi-supervised, or supervised, and it often includes multiple layers of processing, allowing the model to learn increasingly abstract representations of the data.

Overall, machine learning represents a significant advancement in the ability of computers to process information and make informed decisions based on that information.

In [ ]:
query = "What is a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A Convolutional Neural Network (CNN) is a specialized type of Artificial Neural Network (ANN) that is particularly effective for image-driven pattern recognition tasks. CNNs are designed to process data with a grid-like topology, such as images, and they utilize a unique architecture that distinguishes them from traditional ANNs.

### Key Features of CNNs:

1. **Three-Dimensional Neuron Organization**: 
   - In CNNs, neurons are organized in three dimensions: height, width, and depth. This structure allows CNNs to capture spatial hierarchies in images, where the depth dimension corresponds to the number of channels (e.g., RGB for color images).

2. **Layer Types**:
   - CNNs are composed of three main types of layers:
     - **Convolutional Layers**: These layers apply convolution operations to the input, allowing the network to learn spatial hierarchies and features from the images. Each neuron in a convolutional layer connects to a small region of the input, which helps in detecting local patterns.
     - **Pooling Layers**: These layers reduce the spatial dimensions of the input, helping to decrease the computational load and control overfitting by summarizing the features.
     - **Fully-Connected Layers**: These layers connect every neuron in one layer to every neuron in the next layer, typically used at the end of the network to produce the final output.

3. **Functionality**:
   - The input layer holds the pixel values of the image, while the convolutional layers compute outputs based on local regions of the input. The pooling layers then condense these outputs, and the fully-connected layers produce the final classification scores.

4. **Learning Paradigms**:
   - CNNs primarily utilize supervised learning, where the model is trained on labeled data to minimize classification errors. This is crucial for tasks such as image classification, where the goal is to correctly identify the class of an input image.

5. **Architectural Design**:
   - A common practice in CNN architecture is to stack multiple convolutional layers before pooling layers. This enhances feature extraction capabilities and allows the network to learn more complex patterns.

6. **Resource Intensity**:
   - CNNs can be resource-intensive, especially when processing large images. Techniques such as zero-padding and careful management of layer dimensions are often employed to optimize performance and memory usage.

In summary, CNNs are a powerful tool in the field of machine learning, particularly for image analysis and pattern recognition, leveraging their unique architecture to effectively learn and classify visual data.

In [ ]:
query = "How is a resnet better than a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A ResNet (Residual Network) is considered better than a traditional CNN (Convolutional Neural Network) for several reasons, particularly in the context of training deeper architectures and achieving better performance in various tasks. Here are the key advantages of ResNets over standard CNNs:

1. **Degradation Problem Mitigation**: Traditional CNNs often face the degradation problem, where increasing the depth of the network leads to higher training error. ResNets address this issue by introducing shortcut connections that allow gradients to flow more easily during backpropagation. This makes it easier to optimize deeper networks, as the residual learning framework allows the model to learn residual mappings instead of the original unreferenced mappings.

2. **Higher Accuracy with Increased Depth**: ResNets can be significantly deeper than traditional CNNs without suffering from performance degradation. For instance, ResNet architectures with 50, 101, or even 152 layers have been shown to achieve better accuracy compared to shallower networks. The empirical results demonstrate that deeper ResNets can produce substantially better results on datasets like ImageNet and CIFAR-10.

3. **Generalization Performance**: ResNets exhibit good generalization performance across various recognition tasks. The context mentions that replacing VGG-16 with ResNet-101 in the Faster R-CNN framework led to a notable increase in detection metrics on challenging datasets like COCO, indicating that ResNets can generalize better to unseen data.

4. **Architectural Efficiency**: Despite being deeper, ResNets maintain lower computational complexity compared to traditional architectures like VGG-16. For example, a 152-layer ResNet has lower complexity (11.3 billion FLOPs) than VGG-16 (15.3 billion FLOPs), allowing for more efficient training and inference.

5. **Empirical Success in Competitions**: ResNets have achieved top rankings in various competitions, such as ILSVRC and COCO 2015, demonstrating their effectiveness in real-world applications. The context highlights that models based on deep residual networks won first places in several tracks, showcasing their superior performance.

In summary, ResNets improve upon traditional CNNs by effectively addressing the degradation problem, enabling deeper architectures to be trained successfully, achieving higher accuracy, and demonstrating strong generalization capabilities across different tasks.

In [ ]:
query = "What is NLP and its relation to linguistics?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Natural Language Processing (NLP) is a field within Artificial Intelligence that focuses on the interaction between computers and human languages. The primary goal of NLP is to enable computers to automatically understand, interpret, and generate human languages, which includes both writing and speaking. The term "Natural Language" specifically refers to human languages, distinguishing them from programming languages used in computing.

NLP is closely related to linguistics, which is the scientific study of language and its structure. Linguistics provides the foundational theories and frameworks that inform NLP techniques, as understanding the nuances of human language—such as syntax, semantics, and pragmatics—is essential for developing effective NLP applications. By leveraging insights from linguistics, NLP aims to create systems that can process and analyze large amounts of natural language data, facilitating tasks such as translation, sentiment analysis, and conversational agents. 

In summary, NLP is a crucial intersection of artificial intelligence and linguistics, aiming to bridge the gap between human communication and machine understanding.

In [ ]:
query = "What is the difference between AI, ML and DL?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The difference between AI, ML, and DL can be summarized as follows:

### Artificial Intelligence (AI)
- **Definition**: AI refers to the ability of a computer program or machine to think and learn, mimicking human cognition. It encompasses a broad range of technologies and applications aimed at making machines "smart."
- **Origin**: The term "Artificial Intelligence" was coined by John McCarthy in 1955.
- **Functionality**: AI systems can interpret external data, learn from it, and adapt to achieve specific goals. As technology advances, tasks once considered to require intelligence, like optical character recognition, are no longer classified as AI.

### Machine Learning (ML)
- **Definition**: ML is a subfield of AI that focuses on the development of algorithms that allow computers to learn from and make predictions based on data without being explicitly programmed.
- **Functionality**: ML algorithms build models from sample inputs and can make decisions or predictions based on data. It is particularly useful in scenarios where traditional programming is impractical, such as spam filtering and computer vision.

### Deep Learning (DL)
- **Definition**: DL is a specialized subset of machine learning that primarily uses neural networks with multiple layers (multi-layer neural networks) to process data.
- **Functionality**: In deep learning, the information processed becomes increasingly abstract with each added layer, making it particularly effective for complex tasks like speech and image recognition. DL models are inspired by the biological nervous system but differ significantly from the structural and functional properties of human brains.

In summary, AI is the overarching field that includes both ML and DL, with ML being a specific approach within AI that enables learning from data, and DL being a further specialization of ML that utilizes deep neural networks for more complex data processing tasks.

In [ ]:
query = "What is the difference between transformers and vision transformers?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The primary difference between traditional Transformers and Vision Transformers (ViT) lies in their application and input processing methods.

1. **Input Representation**:
   - **Transformers**: In natural language processing (NLP), Transformers operate on sequences of tokens (words) that are typically represented as embeddings. The input is a 1D sequence of these token embeddings.
   - **Vision Transformers (ViT)**: ViT adapts the Transformer architecture for image classification tasks by treating image patches as tokens. An image is divided into fixed-size patches, which are then flattened and linearly embedded into a sequence. This sequence of patch embeddings is fed into the Transformer, similar to how word embeddings are processed in NLP.

2. **Architecture**:
   - **Transformers**: The standard Transformer architecture consists of layers of multi-headed self-attention and feed-forward neural networks, designed to capture relationships and dependencies in sequential data.
   - **Vision Transformers (ViT)**: While ViT retains the core Transformer architecture, it modifies the input to accommodate 2D image data. The model includes additional components such as position embeddings to retain spatial information about the patches, which is crucial for understanding the structure of images.

3. **Performance and Efficiency**:
   - **Transformers**: In NLP, Transformers have become the standard due to their ability to scale and perform well on large datasets, often requiring significant computational resources.
   - **Vision Transformers (ViT)**: ViT has shown that a pure Transformer can achieve competitive results in image classification, often outperforming traditional convolutional neural networks (CNNs) in terms of efficiency and scalability when pre-trained on large datasets. ViT requires substantially fewer computational resources to train compared to state-of-the-art CNNs, making it a promising alternative for image recognition tasks.

In summary, while both architectures utilize the Transformer framework, Vision Transformers adapt the input and processing methods to effectively handle image data, demonstrating significant advantages in performance and resource efficiency in the realm of computer vision.

In [ ]:
query = "How is self-attention important in transformers?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Self-attention is crucial in transformers for several reasons:

1. **Parallelization**: Self-attention mechanisms allow for significant parallelization of computations. Unlike recurrent layers, which require sequential processing, self-attention can process all input positions simultaneously. This leads to faster training times and more efficient use of computational resources.

2. **Learning Long-Range Dependencies**: One of the key advantages of self-attention is its ability to learn long-range dependencies within the input data. In traditional recurrent neural networks (RNNs), the path length between input and output positions can be long, making it difficult to learn relationships between distant elements. Self-attention, on the other hand, connects all positions with a constant number of sequential operations, facilitating the learning of these long-range dependencies.

3. **Computational Complexity**: The computational complexity of self-attention is O(n² · d), where n is the sequence length and d is the representation dimension. This is more efficient than recurrent layers, which have a complexity of O(n · d²). This efficiency becomes particularly beneficial as the sequence length increases.

4. **Flexibility in Input Representation**: Self-attention allows for flexible input representations, as it can weigh the importance of different parts of the input dynamically. This is particularly useful in tasks where the relevance of input elements can vary significantly.

5. **Application in Vision Transformers**: In the context of Vision Transformers (ViT), self-attention mechanisms have been adapted to process images by applying attention to patches of the image rather than individual pixels. This adaptation allows transformers to achieve state-of-the-art results in image recognition tasks, demonstrating the versatility and effectiveness of self-attention beyond traditional NLP applications.

Overall, self-attention enhances the transformer's ability to process data efficiently and effectively, making it a foundational component of the architecture.

In [ ]:
query = "How does a resnet work?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A ResNet, or Residual Network, operates on the principle of residual learning to address the challenges associated with training deep neural networks. Here’s a detailed explanation of how it works:

### Key Concepts of ResNet

1. **Residual Mapping**:
   - Instead of learning the desired underlying mapping \( H(x) \) directly, ResNets focus on learning a residual mapping \( F(x) = H(x) - x \). This means that the network learns the difference between the desired output and the input, which is often easier to optimize.

2. **Shortcut Connections**:
   - ResNets utilize shortcut connections that skip one or more layers. These connections perform identity mapping, allowing the input \( x \) to be added directly to the output of the stacked layers. This can be mathematically represented as:
     \[
     H(x) = F(x) + x
     \]
   - The addition of the input \( x \) helps in mitigating the vanishing gradient problem, making it easier for the network to learn.

3. **Optimization Benefits**:
   - The formulation of \( F(x) + x \) allows the network to push the residual \( F(x) \) towards zero if the identity mapping is optimal. This is generally easier than fitting a complex mapping directly, especially as the depth of the network increases.

### Architecture

- ResNets can be constructed with various depths, such as 18, 34, 50, 101, and even 152 layers. The architecture includes:
  - **Convolutional Layers**: These layers extract features from the input images.
  - **Batch Normalization**: Applied after each convolution to stabilize and accelerate training.
  - **Pooling Layers**: Used for down-sampling the feature maps.
  - **Fully Connected Layers**: At the end of the network for classification tasks.

### Performance

- ResNets have shown significant improvements in accuracy as the depth increases, unlike traditional plain networks, which suffer from higher training errors with increased depth. For instance, a 34-layer ResNet outperforms an 18-layer ResNet, demonstrating that deeper networks can be effectively trained without degradation in performance.

### Empirical Results

- Extensive experiments on datasets like ImageNet and CIFAR-10 have validated the effectiveness of ResNets. They have achieved state-of-the-art results, including winning the ILSVRC 2015 competition with a 152-layer ResNet, which had lower complexity than previous models like VGG-16/19.

In summary, ResNets leverage residual learning and shortcut connections to facilitate the training of very deep networks, overcoming the optimization difficulties that typically arise with increased depth. This architecture has proven to be highly effective in various image recognition tasks.

In [ ]:
query = "What is LangChain?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.